# Texto para audio

In [4]:
import boto3
from botocore.exceptions import BotoCoreError, ClientError
from contextlib import closing
import os
import sys
import subprocess
from tempfile import gettempdir

polly = boto3.client(service_name="polly")

text = "Teste 1, 2, 3, 4"

try:
    response = polly.synthesize_speech(Text=text, OutputFormat="mp3",
                                        VoiceId="Thiago", Engine="neural")
except (BotoCoreError, ClientError) as error:
    print(error)
    sys.exit(-1)

if "AudioStream" in response:
    # Note: Closing the stream is important because the service throttles on the
    # number of parallel connections. Here we are using contextlib.closing to
    # ensure the close method of the stream object will be called automatically
    # at the end of the with statement's scope.
        with closing(response["AudioStream"]) as stream:
           output = os.path.join(gettempdir(), "polly_speech.mp3")

           try:
                with open(output, "wb") as file:
                   file.write(stream.read())
           except IOError as error:
              print(error)
              sys.exit(-1)

else:
    print("Nao foi possível acessar o audio.")
    sys.exit(-1)

# Salvar no S3
try:
    with open(output, "rb") as file:
        boto3.client('s3').upload_fileobj(file, "mainbucketjean", "aws_polly/polly_speech.mp3")
        print("arquivo enviado para o S3")
except (BotoCoreError, ClientError) as error:
    print("Erro ao enviar o arquivo para o S3:", error)
    sys.exit(-1)

arquivo enviado para o S3


# audio para texto

In [10]:
import boto3

transcribe_client = boto3.client("transcribe")

job = "TranscriptionJobTest"

transcribe_client.start_transcription_job(
    TranscriptionJobName=job,
    Media={"MediaFileUri": "s3://mainbucketjean/aws_polly/polly_speech.mp3"},
    MediaFormat="mp3",
    LanguageCode="pt-BR"
)

{'TranscriptionJob': {'TranscriptionJobName': 'TranscriptionJobTest',
  'TranscriptionJobStatus': 'IN_PROGRESS',
  'LanguageCode': 'pt-BR',
  'MediaFormat': 'mp3',
  'Media': {'MediaFileUri': 's3://mainbucketjean/aws_polly/polly_speech.mp3'},
  'StartTime': datetime.datetime(2024, 9, 5, 15, 52, 25, 884000, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2024, 9, 5, 15, 52, 25, 852000, tzinfo=tzlocal())},
 'ResponseMetadata': {'RequestId': '4be48ab7-f748-4ae7-838c-2bba0f693c38',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4be48ab7-f748-4ae7-838c-2bba0f693c38',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '285',
   'date': 'Thu, 05 Sep 2024 18:52:25 GMT'},
  'RetryAttempts': 0}}

In [11]:
status_job = transcribe_client.get_transcription_job(TranscriptionJobName=job)
status_job

{'TranscriptionJob': {'TranscriptionJobName': 'TranscriptionJobTest',
  'TranscriptionJobStatus': 'COMPLETED',
  'LanguageCode': 'pt-BR',
  'MediaSampleRateHertz': 24000,
  'MediaFormat': 'mp3',
  'Media': {'MediaFileUri': 's3://mainbucketjean/aws_polly/polly_speech.mp3'},
  'Transcript': {'TranscriptFileUri': 'https://s3.us-east-1.amazonaws.com/aws-transcribe-us-east-1-prod/905418327636/TranscriptionJobTest/9e388415-415d-45ab-b6e7-36657473d92f/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjENr%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJIMEYCIQCKQkq%2FaXaRMYD2tEXU4UAEIUbQb8XmAuENjrzPlkJVcwIhAJWdGuwE1OUR0OePyM%2FkmD1fGtTZk0SRtOLnbBMwJSQnKrsFCPP%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEQBBoMMjc2NjU2NDMzMTUzIgyhiHv1PvC4nGpNi4EqjwXDnKXL0YbJSs%2BPm5w6rrLdZ5LqzEAL5RBWm4Urh6E2kTQXOvYurpgXoNdMuOAPmJsFkdB22ReDaLWqImw8giiw3kK8shVUCeaxYEAnLbUzT9VEpskt8bwDWqgerQwPU04dWy3zN3NXyNFmE7gb55RJjRdAQ4kSC3wa6o6WdAtuJIXwpU3rIsgFY1wZaQUQU9Poz3rK1JpBshTjbWLIjqhUjmdGo7CwOaH%2FbRMyloYVRj9489ZR991uX42WbSIf6V8Um2f

In [13]:
import requests
import json

response = requests.get(status_job["TranscriptionJob"]["Transcript"]["TranscriptFileUri"])

with open("polly_transcription.json", "wb") as file:
    file.write(response.content)

In [16]:
with open("polly_transcription.json", "r") as file:
    data = json.load(file)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Regex para tirar as pontuações

In [7]:
import string

texto = '    testando? vambora...FFFfF  é isso aí!     '
texto_sem_pontuacao = ''.join(char for char in texto if char not in string.punctuation)

print(texto_sem_pontuacao.lower().strip())

testando vamborafffff  é isso aí


In [10]:
import boto3

s3_client = boto3.client('s3')

response = s3_client.list_objects_v2(
        Bucket='mainbucketjean',
        Prefix='aws_polly'
    )

for obj in response['Contents']:
    file_name = obj['Key']
    if '4e52e7e4b58b361b' in file_name:
        print(file_name.split('/')[-1])

teste_1,_2_4e52e7e4b58b361b.mp3
